In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,LearningRateScheduler,EarlyStopping

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        r = (band_1 - band_1.min()) / (band_1.max() - band_1.min())
        g = (band_2 - band_2.min()) / (band_2.max() - band_2.min())
        b = (band_3 - band_3.min()) / (band_3.max() - band_3.min())

        rgb = np.dstack((r, g, b))
        images.append(rgb)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding='same',input_shape=(75,75,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model
print('model model')


model model


In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    elif epoch<50:
        return 0.0001
    else:
        return 0.00005

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            rotation_range = 20,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.2,
            horizontal_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)

import pickle
with open('../features/cnn_1_aug_rescale_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))

#pre 3219
# new 2965

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.69267, saving model to best_m.h5
 - 3s - loss: 0.7032 - acc: 0.5251 - val_loss: 0.6927 - val_acc: 0.5140
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 1s - loss: 0.6884 - acc: 0.5546 - val_loss: 0.6991 - val_acc: 0.5084
Epoch 3/100
Epoch 00003: val_loss improved from 0.69267 to 0.62700, saving model to best_m.h5
 - 1s - loss: 0.6728 - acc: 0.5735 - val_loss: 0.6270 - val_acc: 0.6785
Epoch 4/100
Epoch 00004: val_loss improved from 0.62700 to 0.60496, saving model to best_m.h5
 - 1s - loss: 0.5988 - acc: 0.6523 - val_loss: 0.6050 - val_acc: 0.4953
Epoch 5/100
Epoch 00005: val_loss improved from 0.60496 to 0.56853, saving model to best_m.h5
 - 1s - loss: 0.5732 - acc: 0.6639 - val_loss: 0.5685 - val_acc: 0.6879
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 1s - loss: 0.5847 - acc: 0.6573 - val_loss: 0.5920 - val_acc: 0.6710
Epoch 7/100
Epoch 00007: val_loss improved from 0.56853 to 0.56143, saving model to best_m.h5
 -

Epoch 60/100
Epoch 00060: val_loss did not improve
 - 1s - loss: 0.3380 - acc: 0.8414 - val_loss: 0.3148 - val_acc: 0.8411
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 1s - loss: 0.3193 - acc: 0.8480 - val_loss: 0.3112 - val_acc: 0.8280
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 1s - loss: 0.3457 - acc: 0.8336 - val_loss: 0.3176 - val_acc: 0.8318
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.3193 - acc: 0.8341 - val_loss: 0.3352 - val_acc: 0.8262
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.3332 - acc: 0.8433 - val_loss: 0.3200 - val_acc: 0.8318
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.3312 - acc: 0.8495 - val_loss: 0.3118 - val_acc: 0.8374
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.3253 - acc: 0.8400 - val_loss: 0.3211 - val_acc: 0.8318
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.3249 - acc: 0.8488 - val_loss: 0.3369 - val_acc: 0.8280
Epoch 68/100
Epo

Epoch 21/100
Epoch 00021: val_loss did not improve
 - 1s - loss: 0.4056 - acc: 0.8087 - val_loss: 0.3721 - val_acc: 0.8280
Epoch 22/100
Epoch 00022: val_loss improved from 0.37186 to 0.34942, saving model to best_m.h5
 - 1s - loss: 0.4001 - acc: 0.8079 - val_loss: 0.3494 - val_acc: 0.8598
Epoch 23/100
Epoch 00023: val_loss improved from 0.34942 to 0.34903, saving model to best_m.h5
 - 1s - loss: 0.3939 - acc: 0.8128 - val_loss: 0.3490 - val_acc: 0.8542
Epoch 24/100
Epoch 00024: val_loss did not improve
 - 1s - loss: 0.4148 - acc: 0.7953 - val_loss: 0.3943 - val_acc: 0.8318
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 1s - loss: 0.3726 - acc: 0.8248 - val_loss: 0.3785 - val_acc: 0.8318
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 1s - loss: 0.3710 - acc: 0.8279 - val_loss: 0.3880 - val_acc: 0.7981
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 1s - loss: 0.3849 - acc: 0.8239 - val_loss: 0.3761 - val_acc: 0.8318
Epoch 28/100
Epoch 00028: val_loss did not improve


Epoch 82/100
Epoch 00082: val_loss did not improve
 - 1s - loss: 0.3126 - acc: 0.8549 - val_loss: 0.3031 - val_acc: 0.8598
Epoch 83/100
Epoch 00083: val_loss did not improve
 - 1s - loss: 0.3270 - acc: 0.8466 - val_loss: 0.3136 - val_acc: 0.8449
Epoch 84/100
Epoch 00084: val_loss did not improve
 - 1s - loss: 0.3111 - acc: 0.8632 - val_loss: 0.3041 - val_acc: 0.8617
Epoch 85/100
Epoch 00085: val_loss did not improve
 - 1s - loss: 0.2995 - acc: 0.8556 - val_loss: 0.3020 - val_acc: 0.8598
Epoch 86/100
Epoch 00086: val_loss did not improve
 - 1s - loss: 0.3388 - acc: 0.8456 - val_loss: 0.3062 - val_acc: 0.8579
Epoch 87/100
Epoch 00087: val_loss did not improve
 - 1s - loss: 0.3154 - acc: 0.8452 - val_loss: 0.3092 - val_acc: 0.8523
Epoch 88/100
Epoch 00088: val_loss did not improve
 - 1s - loss: 0.3246 - acc: 0.8397 - val_loss: 0.3032 - val_acc: 0.8561
Epoch 89/100
Epoch 00089: val_loss did not improve
 - 1s - loss: 0.3065 - acc: 0.8484 - val_loss: 0.3133 - val_acc: 0.8467
Epoch 90/100
Epo

Epoch 41/100
Epoch 00041: val_loss did not improve
 - 1s - loss: 0.3838 - acc: 0.8302 - val_loss: 0.3782 - val_acc: 0.8127
Epoch 42/100
Epoch 00042: val_loss improved from 0.37669 to 0.36589, saving model to best_m.h5
 - 1s - loss: 0.3724 - acc: 0.8228 - val_loss: 0.3659 - val_acc: 0.8202
Epoch 43/100
Epoch 00043: val_loss did not improve
 - 1s - loss: 0.4125 - acc: 0.8052 - val_loss: 0.3806 - val_acc: 0.8090
Epoch 44/100
Epoch 00044: val_loss did not improve
 - 1s - loss: 0.3915 - acc: 0.8267 - val_loss: 0.3715 - val_acc: 0.8146
Epoch 45/100
Epoch 00045: val_loss did not improve
 - 1s - loss: 0.3957 - acc: 0.8094 - val_loss: 0.3907 - val_acc: 0.8127
Epoch 46/100
Epoch 00046: val_loss improved from 0.36589 to 0.35754, saving model to best_m.h5
 - 1s - loss: 0.3779 - acc: 0.8305 - val_loss: 0.3575 - val_acc: 0.8277
Epoch 47/100
Epoch 00047: val_loss did not improve
 - 1s - loss: 0.3653 - acc: 0.8340 - val_loss: 0.3653 - val_acc: 0.8221
Epoch 48/100
Epoch 00048: val_loss did not improve


In [6]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(32, (3, 3),input_shape=(75, 75, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    return model
print('model model')

model model


In [7]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            rotation_range = 20,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.2,
            horizontal_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)


with open('../features/cnn_2_aug_rescale_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))

# this 2728

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.70006, saving model to best_m.h5
 - 2s - loss: 0.6908 - acc: 0.5233 - val_loss: 0.7001 - val_acc: 0.5140
Epoch 2/100
Epoch 00002: val_loss improved from 0.70006 to 0.59484, saving model to best_m.h5
 - 1s - loss: 0.6405 - acc: 0.6197 - val_loss: 0.5948 - val_acc: 0.6916
Epoch 3/100
Epoch 00003: val_loss improved from 0.59484 to 0.56506, saving model to best_m.h5
 - 1s - loss: 0.5786 - acc: 0.6836 - val_loss: 0.5651 - val_acc: 0.6991
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 1s - loss: 0.5745 - acc: 0.6890 - val_loss: 0.6126 - val_acc: 0.6692
Epoch 5/100
Epoch 00005: val_loss did not improve
 - 1s - loss: 0.5399 - acc: 0.7202 - val_loss: 0.5863 - val_acc: 0.6692
Epoch 6/100
Epoch 00006: val_loss improved from 0.56506 to 0.55954, saving model to best_m.h5
 - 1s - loss: 0.5372 - acc: 0.7243 - val_loss: 0.5595 - val_acc: 0.7065
Epoch 7/100
Epoch 00007: val_loss improved from 0.55954 to 0.47674, saving model to best_m.h5
 -

Epoch 61/100
Epoch 00061: val_loss did not improve
 - 1s - loss: 0.2562 - acc: 0.8886 - val_loss: 0.3029 - val_acc: 0.8692
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 1s - loss: 0.2618 - acc: 0.8800 - val_loss: 0.2800 - val_acc: 0.8766
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.2479 - acc: 0.8892 - val_loss: 0.2683 - val_acc: 0.8748
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.2637 - acc: 0.8921 - val_loss: 0.2708 - val_acc: 0.8729
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.2620 - acc: 0.8802 - val_loss: 0.2778 - val_acc: 0.8729
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.2569 - acc: 0.8812 - val_loss: 0.2644 - val_acc: 0.8692
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.2461 - acc: 0.8892 - val_loss: 0.2760 - val_acc: 0.8766
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.2548 - acc: 0.8810 - val_loss: 0.2729 - val_acc: 0.8710
Epoch 69/100
Epo

Epoch 21/100
Epoch 00021: val_loss did not improve
 - 1s - loss: 0.3852 - acc: 0.8246 - val_loss: 0.3319 - val_acc: 0.8598
Epoch 22/100
Epoch 00022: val_loss did not improve
 - 1s - loss: 0.4040 - acc: 0.8173 - val_loss: 0.3365 - val_acc: 0.8561
Epoch 23/100
Epoch 00023: val_loss did not improve
 - 1s - loss: 0.3752 - acc: 0.8312 - val_loss: 0.3506 - val_acc: 0.8505
Epoch 24/100
Epoch 00024: val_loss did not improve
 - 1s - loss: 0.3844 - acc: 0.8277 - val_loss: 0.3534 - val_acc: 0.8336
Epoch 25/100
Epoch 00025: val_loss improved from 0.32912 to 0.32059, saving model to best_m.h5
 - 1s - loss: 0.3874 - acc: 0.8334 - val_loss: 0.3206 - val_acc: 0.8710
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 1s - loss: 0.3896 - acc: 0.8185 - val_loss: 0.3430 - val_acc: 0.8505
Epoch 27/100
Epoch 00027: val_loss improved from 0.32059 to 0.31287, saving model to best_m.h5
 - 1s - loss: 0.3627 - acc: 0.8301 - val_loss: 0.3129 - val_acc: 0.8636
Epoch 28/100
Epoch 00028: val_loss improved from 0.

Epoch 83/100
Epoch 00083: val_loss improved from 0.25942 to 0.25932, saving model to best_m.h5
 - 1s - loss: 0.2683 - acc: 0.8793 - val_loss: 0.2593 - val_acc: 0.8766
Epoch 84/100
Epoch 00084: val_loss did not improve
 - 1s - loss: 0.3020 - acc: 0.8644 - val_loss: 0.2616 - val_acc: 0.8748
Epoch 85/100
Epoch 00085: val_loss did not improve
 - 1s - loss: 0.2866 - acc: 0.8596 - val_loss: 0.2628 - val_acc: 0.8785
Epoch 86/100
Epoch 00086: val_loss did not improve
 - 1s - loss: 0.2862 - acc: 0.8700 - val_loss: 0.2634 - val_acc: 0.8766
Epoch 87/100
Epoch 00087: val_loss improved from 0.25932 to 0.25874, saving model to best_m.h5
 - 1s - loss: 0.2908 - acc: 0.8663 - val_loss: 0.2587 - val_acc: 0.8766
Epoch 88/100
Epoch 00088: val_loss did not improve
 - 1s - loss: 0.2792 - acc: 0.8672 - val_loss: 0.2686 - val_acc: 0.8748
Epoch 89/100
Epoch 00089: val_loss did not improve
 - 1s - loss: 0.2774 - acc: 0.8655 - val_loss: 0.2696 - val_acc: 0.8748
Epoch 90/100
Epoch 00090: val_loss did not improve


Epoch 42/100
Epoch 00042: val_loss did not improve
 - 1s - loss: 0.2744 - acc: 0.8826 - val_loss: 0.2999 - val_acc: 0.8596
Epoch 43/100
Epoch 00043: val_loss did not improve
 - 1s - loss: 0.2658 - acc: 0.8755 - val_loss: 0.3016 - val_acc: 0.8558
Epoch 44/100
Epoch 00044: val_loss did not improve
 - 1s - loss: 0.2908 - acc: 0.8650 - val_loss: 0.3020 - val_acc: 0.8558
Epoch 45/100
Epoch 00045: val_loss did not improve
 - 1s - loss: 0.2823 - acc: 0.8655 - val_loss: 0.3135 - val_acc: 0.8427
Epoch 46/100
Epoch 00046: val_loss improved from 0.29771 to 0.29528, saving model to best_m.h5
 - 1s - loss: 0.2769 - acc: 0.8768 - val_loss: 0.2953 - val_acc: 0.8596
Epoch 47/100
Epoch 00047: val_loss did not improve
 - 1s - loss: 0.2894 - acc: 0.8589 - val_loss: 0.3029 - val_acc: 0.8670
Epoch 48/100
Epoch 00048: val_loss improved from 0.29528 to 0.29499, saving model to best_m.h5
 - 1s - loss: 0.2736 - acc: 0.8674 - val_loss: 0.2950 - val_acc: 0.8727
Epoch 49/100
Epoch 00049: val_loss did not improve


In [8]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')

model model


In [9]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            rotation_range = 20,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.2,
            horizontal_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)


with open('../features/cnn_3_aug_rescale_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))

# this 2737

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.67802, saving model to best_m.h5
 - 3s - loss: 0.6902 - acc: 0.5474 - val_loss: 0.6780 - val_acc: 0.5960
Epoch 2/100
Epoch 00002: val_loss improved from 0.67802 to 0.64642, saving model to best_m.h5
 - 2s - loss: 0.6717 - acc: 0.5447 - val_loss: 0.6464 - val_acc: 0.5087
Epoch 3/100
Epoch 00003: val_loss improved from 0.64642 to 0.58130, saving model to best_m.h5
 - 2s - loss: 0.6218 - acc: 0.6039 - val_loss: 0.5813 - val_acc: 0.6758
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 2s - loss: 0.5842 - acc: 0.6639 - val_loss: 0.6890 - val_acc: 0.6409
Epoch 5/100
Epoch 00005: val_loss improved from 0.58130 to 0.53737, saving model to best_m.h5
 - 2s - loss: 0.5963 - acc: 0.6656 - val_loss: 0.5374 - val_acc: 0.7307
Epoch 6/100
Epoch 00006: val_loss improved from 0.53737 to 0.53272, saving model to best_m.h5
 - 2s - loss: 0.5489 - acc: 0.7108 - val_loss: 0.5327 - val_acc: 0.7556
Epoch 7/100
Epoch 00007: val_loss did not improve
 -

Epoch 60/100
Epoch 00060: val_loss did not improve
 - 2s - loss: 0.3352 - acc: 0.8450 - val_loss: 0.2720 - val_acc: 0.8728
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 2s - loss: 0.3146 - acc: 0.8483 - val_loss: 0.2805 - val_acc: 0.8653
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.3317 - acc: 0.8489 - val_loss: 0.2732 - val_acc: 0.8753
Epoch 63/100
Epoch 00063: val_loss improved from 0.27007 to 0.26889, saving model to best_m.h5
 - 2s - loss: 0.3098 - acc: 0.8491 - val_loss: 0.2689 - val_acc: 0.8703
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.3221 - acc: 0.8522 - val_loss: 0.2711 - val_acc: 0.8653
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.3165 - acc: 0.8483 - val_loss: 0.2764 - val_acc: 0.8678
Epoch 66/100
Epoch 00066: val_loss improved from 0.26889 to 0.26836, saving model to best_m.h5
 - 2s - loss: 0.3263 - acc: 0.8441 - val_loss: 0.2684 - val_acc: 0.8703
Epoch 67/100
Epoch 00067: val_loss improved from 0.

Epoch 00020: val_loss improved from 0.40193 to 0.38741, saving model to best_m.h5
 - 2s - loss: 0.4394 - acc: 0.7947 - val_loss: 0.3874 - val_acc: 0.7955
Epoch 21/100
Epoch 00021: val_loss improved from 0.38741 to 0.38517, saving model to best_m.h5
 - 2s - loss: 0.4624 - acc: 0.7772 - val_loss: 0.3852 - val_acc: 0.7980
Epoch 22/100
Epoch 00022: val_loss did not improve
 - 2s - loss: 0.4033 - acc: 0.8158 - val_loss: 0.4164 - val_acc: 0.8080
Epoch 23/100
Epoch 00023: val_loss did not improve
 - 2s - loss: 0.4067 - acc: 0.7993 - val_loss: 0.4291 - val_acc: 0.7855
Epoch 24/100
Epoch 00024: val_loss did not improve
 - 2s - loss: 0.4055 - acc: 0.8197 - val_loss: 0.3938 - val_acc: 0.8080
Epoch 25/100
Epoch 00025: val_loss improved from 0.38517 to 0.37266, saving model to best_m.h5
 - 2s - loss: 0.4073 - acc: 0.8033 - val_loss: 0.3727 - val_acc: 0.8130
Epoch 26/100
Epoch 00026: val_loss improved from 0.37266 to 0.36628, saving model to best_m.h5
 - 2s - loss: 0.3884 - acc: 0.8192 - val_loss: 0

Epoch 00082: val_loss did not improve
 - 2s - loss: 0.3244 - acc: 0.8522 - val_loss: 0.3225 - val_acc: 0.8454
Epoch 83/100
Epoch 00083: val_loss did not improve
 - 2s - loss: 0.3166 - acc: 0.8547 - val_loss: 0.3163 - val_acc: 0.8554
Epoch 84/100
Epoch 00084: val_loss did not improve
 - 2s - loss: 0.3065 - acc: 0.8633 - val_loss: 0.3086 - val_acc: 0.8529
Epoch 85/100
Epoch 00085: val_loss did not improve
 - 2s - loss: 0.3258 - acc: 0.8547 - val_loss: 0.3138 - val_acc: 0.8479
Epoch 86/100
Epoch 00086: val_loss improved from 0.30808 to 0.30652, saving model to best_m.h5
 - 2s - loss: 0.3046 - acc: 0.8567 - val_loss: 0.3065 - val_acc: 0.8479
Epoch 87/100
Epoch 00087: val_loss did not improve
 - 2s - loss: 0.2989 - acc: 0.8581 - val_loss: 0.3075 - val_acc: 0.8504
Epoch 88/100
Epoch 00088: val_loss did not improve
 - 2s - loss: 0.3160 - acc: 0.8525 - val_loss: 0.3123 - val_acc: 0.8504
Epoch 89/100
Epoch 00089: val_loss did not improve
 - 2s - loss: 0.2961 - acc: 0.8650 - val_loss: 0.3100 - v

Epoch 39/100
Epoch 00039: val_loss did not improve
 - 2s - loss: 0.3804 - acc: 0.8283 - val_loss: 0.3013 - val_acc: 0.8504
Epoch 40/100
Epoch 00040: val_loss did not improve
 - 2s - loss: 0.3374 - acc: 0.8458 - val_loss: 0.3019 - val_acc: 0.8479
Epoch 41/100
Epoch 00041: val_loss did not improve
 - 2s - loss: 0.3456 - acc: 0.8400 - val_loss: 0.2917 - val_acc: 0.8504
Epoch 42/100
Epoch 00042: val_loss did not improve
 - 2s - loss: 0.3420 - acc: 0.8400 - val_loss: 0.2962 - val_acc: 0.8529
Epoch 43/100
Epoch 00043: val_loss did not improve
 - 2s - loss: 0.3523 - acc: 0.8314 - val_loss: 0.3104 - val_acc: 0.8504
Epoch 44/100
Epoch 00044: val_loss did not improve
 - 2s - loss: 0.3541 - acc: 0.8358 - val_loss: 0.2918 - val_acc: 0.8554
Epoch 45/100
Epoch 00045: val_loss did not improve
 - 2s - loss: 0.3522 - acc: 0.8533 - val_loss: 0.3008 - val_acc: 0.8554
Epoch 46/100
Epoch 00046: val_loss did not improve
 - 2s - loss: 0.3408 - acc: 0.8372 - val_loss: 0.3104 - val_acc: 0.8554
Epoch 47/100
Epo

Epoch 3/100
Epoch 00003: val_loss did not improve
 - 2s - loss: 0.6634 - acc: 0.6005 - val_loss: 0.6620 - val_acc: 0.6209
Epoch 4/100
Epoch 00004: val_loss improved from 0.63780 to 0.61830, saving model to best_m.h5
 - 2s - loss: 0.6424 - acc: 0.5986 - val_loss: 0.6183 - val_acc: 0.5137
Epoch 5/100
Epoch 00005: val_loss did not improve
 - 2s - loss: 0.6370 - acc: 0.5945 - val_loss: 0.6354 - val_acc: 0.5062
Epoch 6/100
Epoch 00006: val_loss improved from 0.61830 to 0.56070, saving model to best_m.h5
 - 2s - loss: 0.6024 - acc: 0.6600 - val_loss: 0.5607 - val_acc: 0.7082
Epoch 7/100
Epoch 00007: val_loss did not improve
 - 2s - loss: 0.6299 - acc: 0.6103 - val_loss: 0.5939 - val_acc: 0.5910
Epoch 8/100
Epoch 00008: val_loss improved from 0.56070 to 0.54876, saving model to best_m.h5
 - 2s - loss: 0.5850 - acc: 0.6789 - val_loss: 0.5488 - val_acc: 0.7207
Epoch 9/100
Epoch 00009: val_loss did not improve
 - 2s - loss: 0.5777 - acc: 0.6922 - val_loss: 0.5560 - val_acc: 0.7207
Epoch 10/100
E

Epoch 63/100
Epoch 00063: val_loss did not improve
 - 2s - loss: 0.3047 - acc: 0.8623 - val_loss: 0.3435 - val_acc: 0.8504
Epoch 64/100
Epoch 00064: val_loss improved from 0.32452 to 0.31918, saving model to best_m.h5
 - 2s - loss: 0.3262 - acc: 0.8558 - val_loss: 0.3192 - val_acc: 0.8728
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.3095 - acc: 0.8633 - val_loss: 0.3414 - val_acc: 0.8504
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.3384 - acc: 0.8656 - val_loss: 0.3597 - val_acc: 0.8454
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.3145 - acc: 0.8558 - val_loss: 0.3300 - val_acc: 0.8529
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.3315 - acc: 0.8475 - val_loss: 0.3378 - val_acc: 0.8653
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.3074 - acc: 0.8591 - val_loss: 0.3294 - val_acc: 0.8628
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 2s - loss: 0.3195 - acc: 0.8558 - val_los

In [10]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def ConvBlock(model, layers, filters):
    '''Create [layers] layers consisting of zero padding, a convolution with [filters] 3x3 filters and batch normalization. Perform max pooling after the last layer.'''
    for i in range(layers):
        model.add(ZeroPadding2D((1, 1)))
        model.add(Conv2D(filters, (3, 3), activation='relu'))
        model.add(BatchNormalization(axis=3))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()

    # Input image: 75x75x3
    model.add(Lambda(lambda x: x, input_shape=(75, 75, 3)))
    ConvBlock(model, 1, 32)
    # 37x37x32
    ConvBlock(model, 1, 64)
    # 18x18x64
    ConvBlock(model, 1, 128)
    # 9x9x128
    ConvBlock(model, 1, 128)
    # 4x4x128
    model.add(Flatten())
    model.add(Dense(1,activation='sigmoid'))
    
    return model
print('model model')


model model


In [11]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            rotation_range = 20,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.2,
            horizontal_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)


with open('../features/cnn_4_aug_rescale_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))

# this 2146

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.70834, saving model to best_m.h5
 - 3s - loss: 0.6396 - acc: 0.7225 - val_loss: 0.7083 - val_acc: 0.4860
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 2s - loss: 0.5090 - acc: 0.7742 - val_loss: 0.7711 - val_acc: 0.4860
Epoch 3/100
Epoch 00003: val_loss improved from 0.70834 to 0.70518, saving model to best_m.h5
 - 2s - loss: 0.4735 - acc: 0.7763 - val_loss: 0.7052 - val_acc: 0.5047
Epoch 4/100
Epoch 00004: val_loss improved from 0.70518 to 0.58868, saving model to best_m.h5
 - 2s - loss: 0.4128 - acc: 0.8083 - val_loss: 0.5887 - val_acc: 0.6523
Epoch 5/100
Epoch 00005: val_loss did not improve
 - 2s - loss: 0.4240 - acc: 0.8038 - val_loss: 1.0552 - val_acc: 0.5140
Epoch 6/100
Epoch 00006: val_loss improved from 0.58868 to 0.48924, saving model to best_m.h5
 - 2s - loss: 0.4261 - acc: 0.7882 - val_loss: 0.4892 - val_acc: 0.7794
Epoch 7/100
Epoch 00007: val_loss improved from 0.48924 to 0.37279, saving model to best_m.h5
 -

Epoch 62/100
Epoch 00062: val_loss did not improve
 - 2s - loss: 0.1919 - acc: 0.9170 - val_loss: 0.2683 - val_acc: 0.8804
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 2s - loss: 0.2036 - acc: 0.9183 - val_loss: 0.3324 - val_acc: 0.8654
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 2s - loss: 0.1901 - acc: 0.9193 - val_loss: 0.3077 - val_acc: 0.8654
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 2s - loss: 0.1987 - acc: 0.9034 - val_loss: 0.2873 - val_acc: 0.8766
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 2s - loss: 0.1925 - acc: 0.9243 - val_loss: 0.2953 - val_acc: 0.8673
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 2s - loss: 0.2021 - acc: 0.9172 - val_loss: 0.2954 - val_acc: 0.8710
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 2s - loss: 0.1903 - acc: 0.9233 - val_loss: 0.2646 - val_acc: 0.8710
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 2s - loss: 0.1988 - acc: 0.9117 - val_loss: 0.2804 - val_acc: 0.8710
Epoch 70/100
Epo

Epoch 24/100
Epoch 00024: val_loss did not improve
 - 2s - loss: 0.2853 - acc: 0.8668 - val_loss: 0.3028 - val_acc: 0.8542
Epoch 25/100
Epoch 00025: val_loss did not improve
 - 2s - loss: 0.2956 - acc: 0.8679 - val_loss: 0.3127 - val_acc: 0.8673
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 2s - loss: 0.2799 - acc: 0.8805 - val_loss: 0.2686 - val_acc: 0.8804
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 2s - loss: 0.2629 - acc: 0.8859 - val_loss: 0.3723 - val_acc: 0.8336
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 2s - loss: 0.2575 - acc: 0.8778 - val_loss: 0.5464 - val_acc: 0.7925
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 2s - loss: 0.2699 - acc: 0.8750 - val_loss: 0.3889 - val_acc: 0.8262
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 2s - loss: 0.2974 - acc: 0.8765 - val_loss: 0.3316 - val_acc: 0.8467
Epoch 31/100
Epoch 00031: val_loss improved from 0.26642 to 0.25494, saving model to best_m.h5
 - 2s - loss: 0.2557 - acc: 0.8846 - val_los

Epoch 90/100
Epoch 00090: val_loss did not improve
 - 2s - loss: 0.1528 - acc: 0.9482 - val_loss: 0.2853 - val_acc: 0.8841
Epoch 91/100
Epoch 00091: val_loss did not improve
 - 2s - loss: 0.1814 - acc: 0.9309 - val_loss: 0.2621 - val_acc: 0.8841
Epoch 92/100
Epoch 00092: val_loss did not improve
 - 2s - loss: 0.1727 - acc: 0.9293 - val_loss: 0.2625 - val_acc: 0.8841
Epoch 93/100
Epoch 00093: val_loss did not improve
 - 2s - loss: 0.1713 - acc: 0.9290 - val_loss: 0.2616 - val_acc: 0.8916
Epoch 94/100
Epoch 00094: val_loss did not improve
 - 2s - loss: 0.1627 - acc: 0.9371 - val_loss: 0.2586 - val_acc: 0.8916
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 2s - loss: 0.1697 - acc: 0.9290 - val_loss: 0.2592 - val_acc: 0.8953
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 2s - loss: 0.1329 - acc: 0.9496 - val_loss: 0.2664 - val_acc: 0.8879
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 2s - loss: 0.1614 - acc: 0.9394 - val_loss: 0.2579 - val_acc: 0.8897
Epoch 98/100
Epo

Epoch 52/100
Epoch 00052: val_loss did not improve
 - 2s - loss: 0.1511 - acc: 0.9412 - val_loss: 0.3193 - val_acc: 0.8708
Epoch 53/100
Epoch 00053: val_loss did not improve
 - 2s - loss: 0.1611 - acc: 0.9317 - val_loss: 0.2916 - val_acc: 0.8764
Epoch 54/100
Epoch 00054: val_loss did not improve
 - 2s - loss: 0.1894 - acc: 0.9205 - val_loss: 0.2905 - val_acc: 0.8652
Epoch 55/100
Epoch 00055: val_loss did not improve
 - 2s - loss: 0.1628 - acc: 0.9309 - val_loss: 0.3028 - val_acc: 0.8652
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 2s - loss: 0.1707 - acc: 0.9213 - val_loss: 0.2916 - val_acc: 0.8764
Epoch 57/100
Epoch 00057: val_loss did not improve
 - 2s - loss: 0.1637 - acc: 0.9318 - val_loss: 0.3047 - val_acc: 0.8764
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 2s - loss: 0.1489 - acc: 0.9449 - val_loss: 0.2911 - val_acc: 0.8708
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 2s - loss: 0.1555 - acc: 0.9421 - val_loss: 0.3064 - val_acc: 0.8577
Epoch 60/100
Epo

In [12]:
def create_model():
    '''Create the FCN and return a keras model.'''

    model=Sequential()
    
    # CNN 1
    model.add(Conv2D(64, kernel_size=(3, 3), input_shape=(75, 75, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 2
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.2))

    # CNN 3
    model.add(Conv2D(128, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    #CNN 4
    model.add(Conv2D(64, kernel_size=(3, 3)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    model.add(Dropout(0.3))

    # You must flatten the data for the dense layers
    model.add(Flatten())

    #Dense 1
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.2))

    #Dense 2
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.2))

    # Output 
    model.add(Dense(1, activation="sigmoid"))
    return model
print('model model')


model model


In [13]:
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            rotation_range = 20,
            width_shift_range = 0.1,
            height_shift_range = 0.1,
            zoom_range = 0.2,
            horizontal_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=3)


with open('../features/cnn_5_aug_rescale_preprocess_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
print(log_loss(y,train_pred))
# 2364

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.69023, saving model to best_m.h5
 - 5s - loss: 0.6257 - acc: 0.6595 - val_loss: 0.6902 - val_acc: 0.4860
Epoch 2/100
Epoch 00002: val_loss did not improve
 - 3s - loss: 0.5063 - acc: 0.7243 - val_loss: 0.7069 - val_acc: 0.5159
Epoch 3/100
Epoch 00003: val_loss did not improve
 - 3s - loss: 0.4576 - acc: 0.7948 - val_loss: 1.2516 - val_acc: 0.5140
Epoch 4/100
Epoch 00004: val_loss did not improve
 - 3s - loss: 0.4657 - acc: 0.7756 - val_loss: 0.7621 - val_acc: 0.5178
Epoch 5/100
Epoch 00005: val_loss did not improve
 - 3s - loss: 0.4264 - acc: 0.7889 - val_loss: 0.8569 - val_acc: 0.5159
Epoch 6/100
Epoch 00006: val_loss did not improve
 - 3s - loss: 0.4402 - acc: 0.7850 - val_loss: 0.9748 - val_acc: 0.4879
Epoch 7/100
Epoch 00007: val_loss improved from 0.69023 to 0.62491, saving model to best_m.h5
 - 3s - loss: 0.4119 - acc: 0.8173 - val_loss: 0.6249 - val_acc: 0.6916
Epoch 8/100
Epoch 00008: val_loss did not improve
 - 3s - loss

Epoch 65/100
Epoch 00065: val_loss improved from 0.24624 to 0.24492, saving model to best_m.h5
 - 3s - loss: 0.2937 - acc: 0.8722 - val_loss: 0.2449 - val_acc: 0.8916
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 3s - loss: 0.2700 - acc: 0.8757 - val_loss: 0.2467 - val_acc: 0.8916
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 3s - loss: 0.3035 - acc: 0.8615 - val_loss: 0.2730 - val_acc: 0.8766
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 3s - loss: 0.2967 - acc: 0.8717 - val_loss: 0.2552 - val_acc: 0.8860
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 3s - loss: 0.2853 - acc: 0.8687 - val_loss: 0.2493 - val_acc: 0.8897
Epoch 70/100
Epoch 00070: val_loss did not improve
 - 3s - loss: 0.2521 - acc: 0.8926 - val_loss: 0.2847 - val_acc: 0.8654
Epoch 71/100
Epoch 00071: val_loss did not improve
 - 3s - loss: 0.2675 - acc: 0.8873 - val_loss: 0.2486 - val_acc: 0.8916
Epoch 72/100
Epoch 00072: val_loss did not improve
 - 3s - loss: 0.2980 - acc: 0.8644 - val_los

Epoch 27/100
Epoch 00027: val_loss did not improve
 - 3s - loss: 0.3640 - acc: 0.8225 - val_loss: 0.3866 - val_acc: 0.8037
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 3s - loss: 0.3196 - acc: 0.8577 - val_loss: 0.8729 - val_acc: 0.6692
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 3s - loss: 0.3535 - acc: 0.8350 - val_loss: 0.2716 - val_acc: 0.8766
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 3s - loss: 0.3311 - acc: 0.8433 - val_loss: 0.8260 - val_acc: 0.6262
Epoch 31/100
Epoch 00031: val_loss improved from 0.26631 to 0.23501, saving model to best_m.h5
 - 3s - loss: 0.3071 - acc: 0.8579 - val_loss: 0.2350 - val_acc: 0.8953
Epoch 32/100
Epoch 00032: val_loss did not improve
 - 3s - loss: 0.3141 - acc: 0.8613 - val_loss: 0.2383 - val_acc: 0.8879
Epoch 33/100
Epoch 00033: val_loss did not improve
 - 3s - loss: 0.3089 - acc: 0.8561 - val_loss: 0.2728 - val_acc: 0.8766
Epoch 34/100
Epoch 00034: val_loss did not improve
 - 3s - loss: 0.3210 - acc: 0.8568 - val_los

Epoch 94/100
Epoch 00094: val_loss did not improve
 - 3s - loss: 0.2512 - acc: 0.8911 - val_loss: 0.2586 - val_acc: 0.8841
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 3s - loss: 0.2543 - acc: 0.8759 - val_loss: 0.2513 - val_acc: 0.8804
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 3s - loss: 0.2717 - acc: 0.8759 - val_loss: 0.2432 - val_acc: 0.8897
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 3s - loss: 0.2964 - acc: 0.8692 - val_loss: 0.2702 - val_acc: 0.8804
Epoch 98/100
Epoch 00098: val_loss did not improve
 - 3s - loss: 0.2435 - acc: 0.8878 - val_loss: 0.2543 - val_acc: 0.8860
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 3s - loss: 0.2893 - acc: 0.8658 - val_loss: 0.2567 - val_acc: 0.8841
Epoch 100/100
Epoch 00100: val_loss did not improve
 - 3s - loss: 0.2771 - acc: 0.8788 - val_loss: 0.2445 - val_acc: 0.8822
Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.68580, saving model to best_m.h5
 - 7s - loss: 0.6287 - acc: 0.6527 - val_loss: 0

Epoch 58/100
Epoch 00058: val_loss did not improve
 - 3s - loss: 0.2480 - acc: 0.8805 - val_loss: 0.3075 - val_acc: 0.8783
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 3s - loss: 0.2680 - acc: 0.8883 - val_loss: 0.3086 - val_acc: 0.8801
Epoch 60/100
Epoch 00060: val_loss improved from 0.30131 to 0.29959, saving model to best_m.h5
 - 3s - loss: 0.2790 - acc: 0.8797 - val_loss: 0.2996 - val_acc: 0.8783
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 3s - loss: 0.2609 - acc: 0.8842 - val_loss: 0.3048 - val_acc: 0.8577
Epoch 62/100
Epoch 00062: val_loss did not improve
 - 3s - loss: 0.2341 - acc: 0.8994 - val_loss: 0.3092 - val_acc: 0.8708
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 3s - loss: 0.2734 - acc: 0.8730 - val_loss: 0.3056 - val_acc: 0.8820
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 3s - loss: 0.2725 - acc: 0.8804 - val_loss: 0.3055 - val_acc: 0.8708
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 4s - loss: 0.2381 - acc: 0.8907 - val_los